In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.devices.experimental import TestDevicePythonSim, PlainNumpySimulator

In [2]:
qml.enable_return()

In [3]:
dev = TestDevicePythonSim()

In [4]:
dev_old = qml.device('default.qubit', wires=2)

In [5]:
def tmp_expand_fn(cirq, max_expansion):
    return dev.preprocess(cirq)

def batch_tform(circ):
    return [circ], lambda x: x

dev.batch_execute = dev.execute
dev.batch_transform = batch_tform #dev.preprocess
dev.expand_fn = tmp_expand_fn
dev.shots = None
dev._shot_vector = []

In [6]:
a = qml.numpy.array([1.2, 3.4, 4.5])
ops =[qml.RX(a[0], wires=0),
qml.CNOT(wires=(0,1)),
qml.RY(a[1], wires=1),
qml.RZ(a[2], wires=1)]
m = [qml.expval(qml.PauliX(1))]
     
qs = qml.tape.QuantumScript(ops, m)

In [7]:
rold, gold = dev_old.execute_and_gradients([qs], method="adjoint_jacobian")
rold, gold

([array(0.01951912)],
 [(array(-0.05020614), array(0.07384742), array(-0.09051665))])

In [8]:
r, g = dev.execute_and_gradients([qs])

In [9]:
rold

[array(0.01951912)]

In [10]:
r

[array(0.01951912)]

In [11]:
@qml.qnode(dev_old, diff_method="adjoint")
def cold(a):
    ops =[qml.RX(a[0], wires=0),
    qml.CNOT(wires=(0,1)),
    qml.RY(a[1], wires=1),
    qml.RZ(a[2], wires=1)]
    return qml.expval(qml.PauliX(1))

In [12]:
@qml.qnode(dev, diff_method="adjoint")
def c(a):
    ops =[qml.RX(a[0], wires=0),
    qml.CNOT(wires=(0,1)),
    qml.RY(a[1], wires=1),
    qml.RZ(a[2], wires=1)]
    return qml.expval(qml.PauliX(1))

In [13]:
a = qml.numpy.array([1.2, 3.4, 4.5])

In [14]:
c(a)

[array(0.01951912)]

In [15]:
cold(a)

array(0.01951912)

In [16]:
qml.grad(cold)(a)

array([-0.05020614,  0.07384742, -0.09051665])

In [17]:
qml.grad(c)(a)

array([-0.05020614,  0.07384742, -0.09051665])